In [1]:
from __future__ import print_function

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from ilqr.cost import QRCost, FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']
Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']


In [4]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt)

In [5]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'half_cheetah.xml')
"""
State-Space (name/joint/parameter):
        0- rootx     slider      position (m)
        1- rootz     slider      position (m)
        2- rooty     hinge       angle (rad)
        3- bthigh    hinge       angle (rad)
        4- bshin     hinge       angle (rad)
        5- bfoot     hinge       angle (rad)
        6- fthigh    hinge       angle (rad)
        7- fshin     hinge       angle (rad)
        8- ffoot     hinge       angle (rad)
        9- rootx     slider      velocity (m/s)
        10- rootz     slider      velocity (m/s)
        11- rooty     hinge       angular velocity (rad/s)
        12- bthigh    hinge       angular velocity (rad/s)
        13- bshin     hinge       angular velocity (rad/s)
        14- bfoot     hinge       angular velocity (rad/s)
        15- fthigh    hinge       angular velocity (rad/s)
        16- fshin     hinge       angular velocity (rad/s)
        17- ffoot     hinge       angular velocity (rad/s)
"""

"""
Actuators (name/actuator/parameter):
        - bthigh    hinge       torque (N m)
        - bshin     hinge       torque (N m)
        - bfoot     hinge       torque (N m)
        - fthigh    hinge       torque (N m)
        - fshin     hinge       torque (N m)
        - ffoot     hinge       torque (N m)
"""

dynamics = MujocoDynamics(xml_path, frame_skip = 1, use_multiprocessing = True)
print('dt:', dynamics.dt)

Finished loading process Finished loading process 1799017989
Finished loading process
Finished loading processFinished loading process Finished loading process17991  17992
 17993
Finished loading process
 17998
18013
Finished loading processFinished loading process  18022
18035
Finished loading processdt: 0.01
 18040
Finished loading processFinished loading process  1806018055



In [6]:
def l(x, u, i):
    action_cost = np.square(u).sum()
    vel_cost = 10 * (x[9] - 4) ** 2
    steady_cost = 200 * (x[10] ** 2)
    return action_cost + vel_cost + steady_cost

cost2 = FiniteDiffCost(l, lambda x, i: l(x, np.array([0, 0, 0, 0, 0, 0]), i), 18, 6, use_multiprocessing = True)

Finished loading process 18100Finished loading process
 18103Finished loading process
 18112
Finished loading processFinished loading process  18128Finished loading process
 18133
Finished loading processFinished loading process 18119Finished loading process 
18146 18151
Finished loading process
Finished loading process18160  18168
18177Finished loading process

 18188


In [7]:
N = 200
x0 = dynamics.get_state()
us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
#us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N)
mpc = RecedingHorizonController(x0, ilqr)

In [8]:
t0 = time.time()
J_hist = []
controls = mpc.control(us_init, step_size = 2, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []
for i in range(100):
    print('ITERATION', i, '\n')
    us.append(next(controls)[1])
print('time', time.time() - t0)
us = np.concatenate(us)

ITERATION 0 

iteration 0 accepted 38482.79927484741
iteration 1 failed 38482.79927484742
iteration 2 failed 38482.79927484742
iteration 3 accepted 38458.80402446942
iteration 4 accepted 38429.4418275784
iteration 5 accepted 38124.34474006012
iteration 6 accepted 34263.56972769654
iteration 7 accepted 31577.111863984377
iteration 8 accepted 29897.593857211516
iteration 9 accepted 29757.714712205983
iteration 10 accepted 28442.512846130976
iteration 11 accepted 28023.785778105866
iteration 12 accepted 27721.39026693384
iteration 13 accepted 26414.237201049145
iteration 14 accepted 25413.812758006123
iteration 15 accepted 25041.795906663716
iteration 16 accepted 24672.537615698195
iteration 17 accepted 23583.575677981407
iteration 18 accepted 22465.791740309767
iteration 19 accepted 21426.811173513932
iteration 20 accepted 21220.838136738505
iteration 21 accepted 20969.349743536863
iteration 22 accepted 20687.827088635408
iteration 23 accepted 20368.1644976679
iteration 24 accepted 20182

iteration 202 accepted 9742.074729463719
iteration 203 failed 9742.074729463722
iteration 204 accepted 9738.533669892951
iteration 205 failed 9738.53366989296
iteration 206 accepted 9737.995221399276
iteration 207 failed 9737.99522139928
iteration 208 accepted 9737.96545731158
iteration 209 accepted 9737.081725131939
iteration 210 failed 9737.081725131935
iteration 211 accepted 9734.603653846625
iteration 212 accepted 9733.506738005886
iteration 213 accepted 9726.172032014842
iteration 214 accepted 9712.184665011666
iteration 215 accepted 9700.341867049148
iteration 216 accepted 9692.856167247679
iteration 217 accepted 9689.760364270414
iteration 218 accepted 9679.149329646054
iteration 219 accepted 9668.518527409113
iteration 220 accepted 9660.376735588112
iteration 221 accepted 9656.742222428195
iteration 222 accepted 9650.304204179285
iteration 223 accepted 9647.500007101236
iteration 224 accepted 9646.545924728398
iteration 225 accepted 9640.675805519977
iteration 226 accepted 9627

iteration 405 failed 8964.448982587612
iteration 406 failed 8964.448982587612
iteration 407 accepted 8964.092340323401
iteration 408 accepted 8963.451271263843
iteration 409 failed 8963.451271263839
iteration 410 failed 8963.451271263839
iteration 411 accepted 8963.324373062953
iteration 412 failed 8963.324373062958
iteration 413 accepted 8962.53138169584
iteration 414 failed 8962.531381695851
iteration 415 failed 8962.531381695851
iteration 416 failed 8962.531381695851
iteration 417 accepted 8962.344370640923
iteration 418 accepted 8962.254151930654
iteration 419 failed 8962.254151930654
iteration 420 failed 8962.254151930654
iteration 421 accepted 8962.176724182427
iteration 422 failed 8962.176724182431
iteration 423 failed 8962.176724182431
iteration 424 failed 8962.176724182431
iteration 425 failed 8962.176724182431
iteration 426 failed 8962.176724182431
iteration 427 failed 8962.176724182431


/home/wangjim19/ilqr/ilqr/mujoco_controller.py:167: UserWarning: exceeded max regularization term
  warnings.warn("exceeded max regularization term")


ITERATION 1 

iteration 0 failed 8650.272771487547
iteration 1 failed 8650.272771487547
iteration 2 failed 8650.272771487547
iteration 3 failed 8650.272771487547
iteration 4 accepted 8650.226127231565
iteration 5 accepted 8649.451712502683
iteration 6 failed 8649.45171250268
iteration 7 accepted 8644.353403317711
iteration 8 failed 8644.353403317706
iteration 9 accepted 8637.582585266831
iteration 10 accepted 8637.483460368358
iteration 11 accepted 8636.874314772824
iteration 12 accepted 8636.462169541499
iteration 13 failed 8636.462169541503
iteration 14 failed 8636.462169541503
iteration 15 failed 8636.462169541503
iteration 16 accepted 8636.394688458096
iteration 17 failed 8636.394688458102
iteration 18 converged 8636.388784070792
ITERATION 2 

iteration 0 failed 8301.740732817463
iteration 1 failed 8301.740732817463
iteration 2 failed 8301.740732817463
iteration 3 accepted 8298.061148210656
iteration 4 accepted 8297.276117768783
iteration 5 accepted 8296.832255705216
iteration 6 fa

...

iteration 77 failed 14624.511740405469
iteration 78 failed 14624.511740405469
iteration 79 accepted 14623.95946771349
iteration 80 accepted 14623.613270984297
iteration 81 failed 14623.613270984293
iteration 82 accepted 14621.108581079125
iteration 83 failed 14621.108581079126
iteration 84 accepted 14618.309615325252
iteration 85 failed 14618.30961532525
iteration 86 accepted 14615.794172064545
iteration 87 failed 14615.794172064547
iteration 88 accepted 14608.219746668881
iteration 89 failed 14608.219746668885
iteration 90 accepted 14598.032320394266
iteration 91 failed 14598.032320394264
iteration 92 accepted 14597.85835389681
iteration 93 accepted 14596.863974297217
iteration 94 accepted 14593.994411989996
iteration 95 accepted 14591.546262688165
iteration 96 accepted 14531.837713628998
iteration 97 accepted 14480.036743547622
iteration 98 accepted 14472.5662192591
iteration 99 failed 14472.566219259104
ITERATION 87 

iteration 0 failed 14839.98861407307
iteration 1 failed 14839.988

iteration 83 accepted 14165.621006192478
iteration 84 failed 14165.621006192478
iteration 85 accepted 14165.481187317595
iteration 86 failed 14165.481187317593
iteration 87 accepted 14165.355706827519
iteration 88 failed 14165.355706827513
iteration 89 accepted 14157.23671956668
iteration 90 failed 14157.236719566685
iteration 91 accepted 14156.629999646326
iteration 92 failed 14156.629999646326
iteration 93 accepted 14156.592217083751
iteration 94 failed 14156.592217083751
iteration 95 accepted 14156.125155646037
iteration 96 failed 14156.125155646045
iteration 97 accepted 14156.10163762491
iteration 98 failed 14156.101637624903
iteration 99 accepted 14156.042479078027
ITERATION 89 

iteration 0 failed 14468.199699905537
iteration 1 failed 14468.199699905537
iteration 2 failed 14468.199699905537
iteration 3 failed 14468.199699905537
iteration 4 accepted 14467.756151966945
iteration 5 accepted 14467.466830958068
iteration 6 accepted 14465.323156828521
iteration 7 accepted 14459.8718751

iteration 16 failed 16123.675941039806
iteration 17 failed 16123.675941039806
iteration 18 accepted 16112.95929793024
iteration 19 failed 16112.959297930238
iteration 20 accepted 16110.136434707152
iteration 21 failed 16110.13643470715
iteration 22 accepted 16107.784718064664
iteration 23 failed 16107.78471806467
iteration 24 accepted 16093.4287621118
iteration 25 failed 16093.428762111795
iteration 26 accepted 16089.666191609154
iteration 27 accepted 16023.171223638907
iteration 28 failed 16023.1712236389
iteration 29 failed 16023.1712236389
iteration 30 accepted 16018.057711508915
iteration 31 failed 16018.057711508916
iteration 32 accepted 16012.046847333582
iteration 33 failed 16012.046847333579
iteration 34 failed 16012.046847333579
iteration 35 failed 16012.046847333579
iteration 36 accepted 16010.354223939405
iteration 37 failed 16010.354223939406
iteration 38 accepted 16008.293513756787
iteration 39 failed 16008.293513756791
iteration 40 accepted 16007.972071604385
iteration 41

iteration 13 accepted 15816.899064453935
iteration 14 accepted 15816.27904481464
iteration 15 accepted 15810.816026308428
iteration 16 failed 15810.816026308432
iteration 17 accepted 15801.736357460019
iteration 18 failed 15801.736357460017
iteration 19 failed 15801.736357460017
iteration 20 accepted 15800.036731318532
iteration 21 accepted 15799.991193015107
iteration 22 failed 15799.99119301511
iteration 23 failed 15799.99119301511
iteration 24 accepted 15797.552486471961
iteration 25 failed 15797.552486471961
iteration 26 accepted 15792.140600433928
iteration 27 failed 15792.14060043392
iteration 28 accepted 15791.84765033435
iteration 29 failed 15791.847650334359
iteration 30 accepted 15789.57285763594
iteration 31 failed 15789.57285763594
iteration 32 accepted 15787.726426581836
iteration 33 accepted 15776.470781940481
iteration 34 failed 15776.470781940474
iteration 35 failed 15776.470781940474
iteration 36 accepted 15774.367613376437
iteration 37 failed 15774.367613376435
iterat

iteration 19 failed 15217.559261238615
iteration 20 accepted 15215.946693771599
iteration 21 accepted 15212.798244572512
iteration 22 failed 15212.798244572507
iteration 23 accepted 15211.264231607674
iteration 24 failed 15211.26423160768
iteration 25 failed 15211.26423160768
iteration 26 accepted 15210.88543794146
iteration 27 failed 15210.88543794146
iteration 28 accepted 15209.095519407489
iteration 29 accepted 15206.012326089982
iteration 30 failed 15206.012326089982
iteration 31 accepted 15205.953501251608
iteration 32 failed 15205.953501251613
iteration 33 accepted 15205.777753242992
iteration 34 accepted 15202.2398358909
iteration 35 accepted 15199.196840845296
iteration 36 failed 15199.1968408453
iteration 37 accepted 15188.916340273985
iteration 38 accepted 15174.651257110747
iteration 39 accepted 15166.130230418928
iteration 40 accepted 15118.79801021069
iteration 41 accepted 15075.761005421804
iteration 42 failed 15075.761005421802
iteration 43 failed 15075.761005421802
iter

In [21]:
dynamics.sim.reset()
start = x0
start[0] += 3
dynamics.set_state(start)
print(dynamics.get_state())
video = []
for i in range(us.shape[0]):
    dynamics.step(us[i])
    video.append(dynamics.sim.render(512, 512))
    print(dynamics.get_state()[1], dynamics.get_state()[10], dynamics.get_state()[9])
    

[-8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
-0.0027336270182237547 -0.2733627018223755 -0.025478112936793522
-0.005993908305831214 -0.32602812876074594 -0.025867024522646342
-0.009571524224898657 -0.3577615919067443 -0.013230822724106969
-0.013306477942180588 -0.37349537172819314 0.06042347379271641
-0.016994358896094203 -0.3687880953913617 0.200502161590228
-0.020732425727413573 -0.3738066831319371 0.2754469563961084
-0.024705856244455066 -0.39734305170414935 0.3421938624917443
-0.028780042127590985 -0.4074185883135919 0.36757003299617247
-0.0329276670791845 -0.4147624951593515 0.3197487617903621
-0.03775283358503351 -0.48251665058490106 0.2895878495153073
-0.042146731203789235 -0.43938976187557294 0.43673542006530175
-0.044243800699036606 -0.20970694952473737 0.6012419784349745
-0.046636386082292806 -0.23925853832562027 0.8078962089224818
-0.048097075412572764 -0.14606893302799584 0.6922317789344501
-0.04595315043189379 0.21439249806789762 0.775565639650

-0.10998499363699211 -0.026669352020657146 3.945935398143364
-0.11021267946934525 -0.022768583235314253 4.020225539196711
-0.11040680708638993 -0.0194127617044678 4.070858212042155
-0.1107425512620979 -0.033574417570796204 4.100235186793917
-0.11126150620540697 -0.0518954943309078 4.152862313780729
-0.11103690841278047 0.022459779262650317 4.229206898791319
-0.11103836055391686 -0.00014521411363937337 4.274635484502037
-0.11118194667373293 -0.014358611981607698 4.03695402369919
-0.11123885907281936 -0.0056912399086424936 3.9633006374447226
-0.11091045973157697 0.03283993412423885 3.991222158000391
-0.11093564020605944 -0.002518047448247275 4.057687689976214
-0.1105104368096301 0.042520339642932704 4.146567424457356
-0.10981349104033038 0.06969457692997219 4.3925108990942014
-0.10982122377193532 -0.000773273160493318 4.092335984211107
-0.10989263049612558 -0.0071406724190261305 4.002457629202203
-0.11020293201727653 -0.03103015211509585 3.99634911834551
-0.11131001610127285 -0.110708408

In [22]:
make_video_fn(video)()